In [1]:
import os
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

os.environ['WDM_LOG_LEVEL'] = '0'

In [2]:
def inicializarDriver(name):
    if name == ("google-chrome.desktop" or "brave-browser.desktop" or "chromium_chromium.desktop"):
        options = ChromeOptions()
        options.add_argument("--headless")

        if name == "google-chrome.desktop":
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
        elif name == "brave-browser.desktop":
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()), options=options)
        elif name == "chromium_chromium.desktop":
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install()), options=options)

    elif (name == "firefox.desktop"):
        options = FirefoxOptions()
        options.add_argument("--headless")
        driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install(), log_path=os.devnull), options=options)

    return driver

In [3]:
browser_name = ["google-chrome.desktop", "brave-browser.desktop", "chromium_chromium.desktop"]

i = 0

while (i < 3):
    try:
        driver = inicializarDriver(browser_name[i])
        i = 3
    except:
        i += 1

driver.set_page_load_timeout(100)

url = "http://www3.saude.ba.gov.br/cgi/deftohtm.exe?sinan/deng.def"
# url = "http://www3.saude.ba.gov.br/cgi/deftohtm.exe?sinan/chikun.def"
# url = "http://www3.saude.ba.gov.br/cgi/deftohtm.exe?sinan/zika.def"
driver.get(url)

WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/center/div/form/div[4]/div[2]/div[2]/input[1]')))

# Linha
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[2]/div/div[1]/select/option[1]').click()
# Coluna
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[2]/div/div[2]/select/option[6]').click()
# Periodo
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[3]/div/select/option[2]').click()
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[3]/div/select/option[1]').click()

# Mostra
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[4]/div[2]/div[2]/input[1]').click()


WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div[3]/table/tbody/tr/td[1]/a')))

html = driver.page_source

driver.quit()

In [4]:
soup = BeautifulSoup(html, 'html.parser')

tabdados = soup.select(".tabdados tbody tr td ")

tabdados = list(map(lambda node: node.get_text().strip(), tabdados))

col_tabdados = soup.select(".tabdados th ")
col_tabdados = list(map(lambda node: node.get_text().strip(), col_tabdados))

In [5]:
tam_lin = int(len(tabdados)/len(col_tabdados))
tam_col = len(col_tabdados)

aux = []
for i in range (tam_lin):
    aux_2 = []
    for j in range (tam_col):  
        aux_2.append(tabdados.pop(0))
    aux.append(aux_2)

df = pd.DataFrame(aux, columns=col_tabdados)

In [6]:
df = df.drop(columns=['Total'])
df = df.drop(0)
df = df.reset_index()
df = df.drop(columns=['index'])

In [7]:
aux = list(df.iloc[:, 0])
for i in range (len(aux)):
    aux[i] = aux[i].split(" ")
    while (len(aux[i]) > 2):
        aux[i][1] = aux[i][1] + " " + aux[i][2]
        aux[i].pop(2)
aux = pd.DataFrame(aux, columns=["Codigo", "Município"])
df.iloc[:, 0] = aux["Município"]

In [8]:
colunas = [ df["Em Branco"], df["Ignorado"], df["Masculino"], df["Feminino"] ]

for i in range (len(colunas)):
    for j in range (len(colunas[i])):
        try:
            colunas[i][j] = int(colunas[i][j])
        except:
            colunas[i][j] = 0

In [9]:
df = pd.DataFrame(list(zip(df["Município"], colunas[0], colunas[1], colunas[2], colunas[3])), columns = ['Município', 'Em Branco', 'Ignorado', 'Masculino', 'Feminino'])

In [10]:
df.to_csv("../scr/sexo_municipio.csv", encoding='utf-8')

In [11]:
df

,Município,Em Branco,Ignorado,Masculino,Feminino
0,Abaíra,0,0,1,0
1,Abaré,0,0,0,2
2,Acajutiba,0,0,1,2
3,Adustina,0,0,4,5
4,Água Fria,0,0,1,0
...,...,...,...,...,...
345,Vitória da Conquista,0,10,335,422
346,Wagner,0,0,1,6
347,Wanderley,0,0,0,1
348,Wenceslau Guimarães,0,0,1,1
